# Hartree Fock State Energy
ハートリーフォック近似によって得られるエネルギーの値を，第二量子化の方法を使ってpythonで計算する．

In [2]:
from qulacs import *
from qulacs.gate import *
from qulacs.state import *
from openfermion.chem import MolecularData
from openfermion.transforms import get_fermion_operator, jordan_wigner
from openfermionpyscf import run_pyscf
from pyscf import fci

In [3]:
## basis,multiplicity,charge,geometry(原子種と原子の位置),descriptionを定義する．
basis = "sto-3g"
multiplicity = 1
charge = 0
distance  = 0.7414 #Å単位
geometry = [["H", [0,0,0]],["H", [0,0,distance]]]
description  = "tmp"

In [4]:
## OpenFermionのmolecular-dataを読み取る
molecule = MolecularData(geometry, basis, multiplicity, charge, description) #openfermionのMolecularDataをもとに
molecule = run_pyscf(molecule,run_scf=1,run_fci=1)

## ハミルトニアンを生成する．
fermionic_hamiltonian = get_fermion_operator(molecule.get_molecular_hamiltonian())
jw_hamiltonian = jordan_wigner(fermionic_hamiltonian)

## ハミルトニアンをQulacsに使える形に直す
from qulacs import Observable
from qulacs.observable import *
qulacs_hamiltonian = create_observable_from_openfermion_text(str(jw_hamiltonian))

nqubit,nelectron = molecule.n_qubits,molecule.n_electrons
print(nqubit,nelectron)

4 2


In [5]:
### Hartree Fock State
def HF_state(nqubit,nelectron):
    state = QuantumState(nqubit)
    circuit = QuantumCircuit(nqubit)
    for i in range(nelectron):
        circuit.add_gate(X(i))
    circuit.update_quantum_state(state)
    return state

hf_energy = qulacs_hamiltonian.get_expectation_value(HF_state(nqubit,nelectron))
print("HF Energy: {} Hartree".format(hf_energy))

### FCI
exact_energy = molecule.fci_energy
print("Exact Energy: {} Hartree".format(exact_energy))

HF Energy: -1.116684387085341 Hartree
Exact Energy: -1.1372701746609035 Hartree


## 結果
H$_2$ 結合距離0.7414Å HF Energy:-1.116684387085341 Hartree,Exact Energy:-1.1372701746609035 Hartree<br>
N$_2$ 結合距離1.0977Å HF Energy:-107.49589330783462 Hartree,Exact Energy:-107.4958933009819 Hartree<br>
O$_2$ 結合距離1.2075Å HF Energy:-147.55109386392724 Hartree,Exact Energy:-147.74403543362024 Hartree